# Creating a Chatbot linked to the Database

## 1. Importing libraries

In [1]:
%pip install surrealdb==1.0.3 openai==1.55.2 python-dotenv==1.0.1

Note: you may need to restart the kernel to use updated packages.


In [4]:
from surrealdb import Surreal
from dotenv import load_dotenv
from openai import OpenAI
import os

## 2. Connect to the database

In [5]:
load_dotenv(override=True)

db = Surreal(os.getenv("SURREAL_URL"))
db.signin({"username": os.getenv("SURREAL_USER"), "password": os.getenv("SURREAL_PASS")})
db.use(os.getenv("SURREAL_NAMESPACE"), os.getenv("SURREAL_DATABASE"))

print(db.version())

surrealdb-2.2.0


## 4. Embed articles and save to database

In [13]:
client = OpenAI() # api key loaded from .env
embed_model_name = "text-embedding-3-small"
llm_model_name = "gpt-4o"
system_prompt = """
You are an Algerian legal expert AI trained to provide accurate answers.
You must strictly base your answer on the provided context.
You must avoid at all cost to mention in your answer that you are being provided with context.
You must strictly answer in Arabic.
Translate the keywords from context to Arabic.
Your response must adhere to any requested format in the question, if present.
Interpret the context to extract required information and present it directly, without mentioning the reasoning process.
If not, state clearly that you are unable to answer the question.
Do not fabricate or assume facts.
""".strip()

def search_articles(query):
    response = client.embeddings.create(
        input=query,
        model=embed_model_name,
    )
    q_vector = response.data[0].embedding

    results = db.query("""
        SELECT
            text,
            vector::similarity::cosine(embed_vector, $q_vector) AS score
        FROM articles
        ORDER BY score DESC
        LIMIT 5
        """,
        {"q_vector": q_vector}
    )

    return results

def ask_question(query):
    articles = search_articles(query)

    user_prompt = "```context\n"
    for article in articles:
        user_prompt += f"{article['text']}\n"
    user_prompt += "```\n"
    user_prompt += f"```question\n{query}\n```"

    print(user_prompt)
    print("\n------------------\n")

    stream = client.chat.completions.create(
        model=llm_model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        stream=True,
    )
    
    for chunk in stream:
        text_chunk = chunk.choices[0].delta.content
        if text_chunk is None:
            continue
        print(text_chunk, end="")

question = "if I'm christian, what should I do in Algeria? and give me brief answer"
# question = "Anything about family law"
# question = "معلومات عن الانتخابات"
ask_question(question)

```context
المادة 2 : الإسلام  دين الدّولة.
المادة 10 : لا يجوز للمؤسّسات أن تقوم بما يأتي : - الممارسات الإقطاعيّة، والجهويّة، والمحسوبيّة، - إقامة علاقات الاستغلال والتّبعيّة، - السّلوك المخالف للخُلُق الإسلاميّ وقيم ثورة نوفمبر.
المادة 76  : على كلّ مواطن أن يؤدّي بإخلاص واجباته تجاه المجموعة الوطنيّة. التزام المواطن إزاء الوطن وإجباريّة المشاركة في الدّفاع عنه، واجبان مقدّسان دائمان. تضمن الدّولة احترام رموز الثّورة، وأرواح الشّهداء، وكرامة ذويهم، والمجاهدين. وتعمل كذلك على ترقية كتابة التاريخ وتعليمه للأجيال الناشئة.
المادة 30 : الجزائر متضامنة مع جميع الشّعوب الّتي تكافح من أجل التّحرّر السّياسيّ والاقتصاديّ، والحقّ             في تقرير المصير، وضدّ كلّ تمييز عنصريّ.
المادة 31 : تعمل الجزائر من أجل دعم التّعاون الدّوليّ، وتنمية العلاقات الودّيّة بين الدّول، على أساس المساواة، والمصلحة المتبادلة، وعدم التّدخّل في الشّؤون الدّاخليّة. وتتبنّى مبادئ ميثاق الأمم المتّحدة وأهدافه.
```
```question
if I'm christian, what should I do in Algeria? and give me brief answer
```

-------------